In [1]:
import json
import gzip
import numpy as np
import pandas as pd

### Load all pages

In [2]:
with gzip.open('all_pages.json.gz', 'rt', encoding='UTF-8') as zipfile:
    data = json.load(zipfile)

### Extract all titles without words in brackets

In [3]:
def newtitles(data):
    new_titles = []

    for i in range(0, len(data)):
        
        title = data[i]['title']
        new_title = ''
        word = ''
        j = 0
        braket = False 
        
        while j < len(title):
            char = title[j]
            
            if char == ')': 
                braket = False
                word = ''
            
            if char == '(':
                braket = True
                if len(word) > 2:
                    new_title = new_title + word.lower() + ' '
                    word = ''
                    
                
            elif braket: braket = True
                
            elif char.isalpha(): word = word + char
                
            elif char == ' ' or char == '-' or char == '\'':
                if len(word) > 1: 
                    new_title = new_title + word.lower() + ' '
                word = ''
                
            j = j + 1
            
        if len(word) > 2:
            new_title = new_title + word.lower() + ' '
            
        new_titles.append(new_title)
    
    return new_titles

In [4]:
newT = newtitles(data)

### Creation of names (first + last word or single word)

In [5]:
def names(data):
    names = []
    for i in range(len(data)):
        name = []
        word = ''
        for j in range(len(data[i])):
            char = data[i][j]
            if char == ' ':
                name.append(word)
                word = ''
            else: word = word + char
        if len(name) > 1:
            names.append(name[0] + ' '+ name[-1])
        elif len(name) == 1:
            names.append(name[0])
        else: names.append('')

    return names    

In [6]:
names = names(newT)

### Creation of data Frame with new titles

In [7]:
dF = pd.DataFrame(data)
dF['new_title'] = newT
dF['name'] = names

### Find all names that are dublicated

In [8]:
def dublications(dF, col):
    #delete rows with new_title = ''
    cleaned_dF = dF[dF[col] != '']
    #search dublications in new_title and keep those rows
    dubli_dF = cleaned_dF[cleaned_dF.duplicated(subset = col, keep = False)]
    #return the dubli_dF which is sorted by new_title
    return dubli_dF.sort_values(by = col)

In [9]:
name_dub_dF = dublications(dF, 'name')

### Make list of homonyms and disambiguation page names

In [33]:
def homonyms(dF, col):
    h_list = []
    homonyms = []
    dis_names = [dF.iloc[0]['name']]
    homonym = dF.iloc[0][col]
    for i in range(len(dF)):
        if dF.iloc[i][col] == homonym:
            homonyms.append(dF.iloc[i]['title'])
        else:
            h_list.append(homonyms)
            homonyms = [dF.iloc[i]['title']]
            dis_names.append(dF.iloc[i]['name'])
            homonym = dF.iloc[i][col]
    h_list.append(homonyms)
    
    return h_list, dis_names

In [34]:
name_homonomys, dis_names = homonyms(name_dub_dF, 'name')

In [47]:
filearray_names = np.array(name_homonomys, dtype = "object")
np.save('name_homonomys.npy', filearray_names)
filearray_dis = np.array(dis_names, dtype = "object")
np.save('disambiguation_names.npy', filearray_dis)